This tutorial gives an introduction on how to write a bridge (inspired from `test/nonnegative_bridge.jl`).

Bridges are defined entirely in the MathOptInterface world, no JuMP variable, expressions or references are involved.
We start by defining a new set.

In [1]:
using MathOptInterface
const MOI = MathOptInterface
const MOIB = MOI.Bridges

"""
    Nonnegative <: MOI.AbstractScalarSet

Scalar set of nonnegative numbers.
"""
struct Nonnegative <: MOI.AbstractScalarSet end

Nonnegative

The set can already be used in JuMP but no solver supports it.

In [2]:
using JuMP
using GLPK
model = direct_model(MOI.Bridges.full_bridge_optimizer(GLPK.Optimizer(), Float64))
@variable(model, x)
@constraint(model, x in Nonnegative())

ErrorException: Constraints of type MathOptInterface.SingleVariable-in-Nonnegative are not supported by the solver.

This constraint can easily be bridged either into `MOI.SingleVariable`-in-`MOI.GreaterThan` or `MOI.VectorOfVariables`-in-`MOI.Nonnegatives`.
Let's bridge it to `MOI.SingleVariable`-in-`MOI.GreaterThan`.

To create the bridge we first need to figure out what are the constraints and variables that are created.
In this case, no variable is created, only a constraint so we just need to store the index of this constraint.

In [3]:
"""
    NonnegativeBridge{T}

The `NonnegativeBridge` replaces a constraint `func`-in-`Nonnegative` into
`func`-in-`GreaterThan{T}`.
"""
struct NonnegativeBridge{T, F<:MOI.AbstractScalarFunction} <: MOIB.AbstractBridge
    constraint_index::MOI.ConstraintIndex{F, MOI.GreaterThan{T}}
end

NonnegativeBridge

Now we need to implement the creation of the bridge. The constructor transforms the constraint, creates the required variables and constraints in the model and return the bridge containing the indices of the created variables and/or constraints (they are needed later, every index is at least needed for `MOI.delete`).

In [4]:
function NonnegativeBridge{T, F}(model::MOI.ModelLike, f::F, s::Nonnegative) where {T, F}
    ci = MOIU.add_scalar_constraint(model, f, MOI.GreaterThan(zero(T)))
    return NonnegativeBridge{T, F}(ci)
end

In the `LazyBridgeOptimizer` used by JuMP to determine which bridges to use for a given constraint, a Bellman-Ford-like algorithm is used to find the shorted paths (in terms of number of bridges used) from an unsupported constraint to a constraint natively supported.
The bridges plays the role of multi-edges and constraints play the role of nodes.

In [5]:
@constraint(model, x in Nonnegative())

ErrorException: Constraints of type MathOptInterface.SingleVariable-in-Nonnegative are not supported by the solver.

To define the multi-edges of a bridge, the following three methods need to be defined.

In [6]:
function MOI.supports_constraint(::Type{NonnegativeBridge{T}},
                                 ::Type{<:MOI.AbstractScalarFunction},
                                 ::Type{Nonnegative}) where T
    return true
end
function MOIB.added_constraint_types(::Type{NonnegativeBridge{T, F}}) where {T, F}
    return [(F, MOI.GreaterThan{T})]
end
function MOIB.concrete_bridge_type(::Type{NonnegativeBridge{T}},
                                   F::Type{<:MOI.AbstractScalarFunction},
                                   ::Type{Nonnegative}) where T
    # In the constructor, the function `f` of type `F` is passed to
    # `MOIU.add_scalar_constraint` which removes the constrant from `f` but
    # does not change its type so the type of the function in `MOI.GreaterThan`
    # will also be `F`.
    return NonnegativeBridge{T, F}
end

Note that `F` is the type of the function of the constraint created by the bridge and stored in the bridge object,
it is not necessarily equal to the type of the function of the constraint that is bridged, even if it is equal in this example.
The type of the function can be computed using `MOIU.promote_operation`.
Note that `MOIU.promote_operation` does not rely on inference (unlike `Base.promote_op`) so it is safe to use.

For the bridge to be automatically added when a constraint involving `Nonnegative` is added to a model, we can
implement the following method.
Note that this is now a JuMP extension, we leave the MOI world.

In [7]:
function JuMP.build_constraint(_error::Function, func::JuMP.AbstractJuMPScalar, set::Nonnegative)
    return BridgeableConstraint(ScalarConstraint(func, set), NonnegativeBridge)
end

The constraint can now be added.

In [8]:
cref = @constraint(model, x in Nonnegative())

x ∈ Nonnegative()

When the user queries the number of constraints and the list of constraints, he should not see the constraints created by this bridge.

In [9]:
num_constraints(model, VariableRef, MOI.GreaterThan{Float64})

1

In [10]:
all_constraints(model, VariableRef, MOI.GreaterThan{Float64})

1-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.SingleVariable,MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 x ≥ 0.0

Implementing these methods allows the `LazyBridgeOptimizer` to remove them from what is returned by the model.

In [11]:
function MOI.get(::NonnegativeBridge{T, F},
                 ::MOI.NumberOfConstraints{F, MOI.GreaterThan{T}}) where {T, F}
    return 1
end
function MOI.get(bridge::NonnegativeBridge{T, F},
                 ::MOI.ListOfConstraintIndices{F, MOI.GreaterThan{T}}) where {T, F}
    return [bridge.constraint_index]
end

In [12]:
num_constraints(model, VariableRef, MOI.GreaterThan{Float64})

0

In [13]:
all_constraints(model, VariableRef, MOI.GreaterThan{Float64})

0-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.SingleVariable,MathOptInterface.GreaterThan{Float64}},ScalarShape},1}

The primal and dual values need to be transformed. In this case, no transformation is necessary.

In [14]:
optimize!(model)
value(cref)

ArgumentError: ArgumentError: Constraint bridge of type `NonnegativeBridge{Float64,MathOptInterface.SingleVariable}` does not support accessing the attribute `MathOptInterface.ConstraintPrimal(1)`.

In [15]:
function MOI.get(model::MOI.ModelLike,
                 attr::Union{MOI.ConstraintPrimal, MOI.ConstraintDual},
                 bridge::NonnegativeBridge)
    return MOI.get(model, attr, bridge.constraint_index)
end

In [16]:
value(cref)

0.0

In [17]:
dual(cref)

0.0

Now the constraint cannot be deleted unless we implement `MOI.delete` for the bridge.

In [18]:
function MOI.delete(model::MOI.ModelLike, bridge::NonnegativeBridge)
    MOI.delete(model, bridge.constraint_index)
end

In [19]:
delete(model, cref)